<a href="https://colab.research.google.com/github/Glasiermedic/Emergency-Services-Posting-Model/blob/master/socrata_api_to_GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# storing/updating the data using socrata API and BigQuery 

# TASKS
# create google cloud side of project for secret and storage bucket
# define the window of time to request when calling on API
# determine need for authentication with the public dataset (future proof)
# adapt portions of code to be more universal with socrate API ( create class)



import requests
import os
import json
import base64
from datetime import date, timedelta
from sodapy import Socrata
from google.cloud import storage

storage_client = storage.Client()
SECRET_BUCKET = os.environ['SECRET_BUCKET']
STORAGE_BUCKET = os.environ['STORAGE_BUCKET']

def load_creds():
    blob = storage_client.get_bucket(SECRET_BUCKET) \
        .get_blob('creds.json') \
        .download_as_string()

    parsed = json.loads(blob)

    apptoken = parsed['apptoken']
    username = parsed['username']
    password = parsed['password']

    return apptoken, username, password

def cincin_fire(from_when):
   client = Socrata("data.cincinnati-oh.gov", 
                  "rTR5opir2rQWQPBAOk7rNObUI",
                  username= 'rolfson.data@gmail.com', 
                  password="Sfd97dex!5")

    # First 500,000 results, returned as JSON from API / converted to Python list of
    # dictionaries by sodapy.
    results = client.get("vnsz-a3wp") 
    return(results) 

def fetch_and_write(data, context):
    action = base64.b64decode(data['data']).decode('utf-8')
    #window = date.today() - timedelta(1)
    #window = str(window)

    if (action == "download!"):
        payload = cincin_fire(yesterday) 
        payload = '\n'.join(json.dumps(item) for item in payload)

        file_name = "cinfire_{}.json".format(yesterday.replace("-", ""))
        storage_client.get_bucket(STORAGE_BUCKET) \
            .blob(file_name) \
            .upload_from_string(payload)
    else:
        print("No instructions received.")